In [3]:
import violation_common
from copy import deepcopy
from importlib import reload
from scipy.stats import chi2_contingency, kruskal
import pandas as pd
import numpy as np
import pingouin as pg

In [4]:
reload(violation_common)

<module 'violation_common' from '/home/zack/work/NIOSH-Project/violation_common.py'>

In [4]:
raw = violation_common.retreive_raw_violation_data()

In [104]:
raw_copy = deepcopy(raw)

In [105]:
raw_copy = raw[list(set(raw.columns) - {'CONTROLLER_ID', 'VIOLATOR_ID', 'CONTRACTOR_ID', 'MINE_ID'})]

In [106]:
processed = violation_common.process_raw_violation_data(raw_copy)

/home/zack/work/NIOSH-Project/violation_common.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violation_data['MINE_TYPE'].fillna('Facility', inplace=True)
/home/zack/work/NIOSH-Project/violation_common.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violation_data['COAL_METAL_IND'].fillna('M', inplace=True)
/home/zack/work/NIOSH-Project/violation_common.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  violation_data['PRI

In [107]:
processed

,PRIMARY_OR_MILL,VIOLATOR_INSPECTION_DAY_CNT,COAL_METAL_IND,PROPOSED_PENALTY,VIOLATOR_VIOLATION_CNT,MINE_TYPE,SIG_SUB,VIOLATOR_TYPE_CD,YEAR_OCCUR
0,Not_Applicable,0.0,C,475.0,0.0,Underground,Y,Operator,2002.0
1,Not_Applicable,0.0,C,872.0,0.0,Surface,Y,Operator,2001.0
2,Not_Applicable,0.0,C,259.0,0.0,Underground,N,Operator,2002.0
3,Not_Applicable,0.0,C,277.0,0.0,Underground,Y,Operator,2001.0
4,Not_Applicable,0.0,C,150.0,0.0,Surface,Y,Operator,2002.0
...,...,...,...,...,...,...,...,...,...
2792268,Not_Applicable,412.0,C,159.0,166.0,Underground,N,Operator,2021.0
2792269,Primary,170.0,M,5699.0,217.0,Surface,Y,Operator,2023.0
2792270,Primary,2.0,M,133.0,3.0,Surface,N,Operator,2022.0
2792271,Primary,2.0,M,125.0,2.0,Surface,N,Operator,2021.0


In [108]:
processed_clean = processed.dropna()

In [109]:
after_2010 = processed_clean.query('YEAR_OCCUR > 2010')

In [5]:
after_2010 = pd.read_csv('after_2010.csv')

In [8]:
after_2010

,PRIMARY_OR_MILL,VIOLATOR_INSPECTION_DAY_CNT,COAL_METAL_IND,PROPOSED_PENALTY,VIOLATOR_VIOLATION_CNT,MINE_TYPE,SIG_SUB,VIOLATOR_TYPE_CD,YEAR_OCCUR
0,Not_Applicable,2847.0,C,100.0,410.0,Underground,N,Operator,2011.0
1,Not_Applicable,596.0,C,392.0,365.0,Underground,N,Operator,2011.0
2,Not_Applicable,22.0,C,873.0,0.0,Surface,Y,Operator,2011.0
3,Not_Applicable,11.0,C,100.0,0.0,Facility,N,Operator,2011.0
4,Primary,127.0,M,946.0,284.0,Facility,N,Operator,2011.0
...,...,...,...,...,...,...,...,...,...
1262992,Not_Applicable,552.0,C,155.0,179.0,Underground,N,Operator,2022.0
1262993,Not_Applicable,412.0,C,159.0,166.0,Underground,N,Operator,2021.0
1262994,Primary,170.0,M,5699.0,217.0,Surface,Y,Operator,2023.0
1262995,Primary,2.0,M,133.0,3.0,Surface,N,Operator,2022.0


In [17]:
np.std(after_2010['PROPOSED_PENALTY']) / np.mean(after_2010['PROPOSED_PENALTY'])

4.721410093785144

In [19]:
np.unique(after_2010['SIG_SUB'], return_counts=True)

(array(['N', 'Y'], dtype=object), array([969911, 293086]))

In [21]:
293086 / (969911 + 293086)

0.23205597479645637

In [22]:
categorical_cols = ['PRIMARY_OR_MILL', 'COAL_METAL_IND', 'MINE_TYPE', 'VIOLATOR_TYPE_CD']
chi2_results = dict()
for col in categorical_cols:
    contingency_table = pd.crosstab(after_2010['SIG_SUB'], after_2010[col])
    observed = contingency_table.values
    n = contingency_table.sum().sum()
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    V = np.sqrt(chi2 / (n * (min(observed.shape) - 1)))
    chi2_results[col] = {'p-value': p_value, 'V': V, 'stat': chi2}

In [98]:
numerical_cols = ['VIOLATOR_INSPECTION_DAY_CNT', 'VIOLATOR_VIOLATION_CNT', 'YEAR_OCCUR']
kruskal_results = dict()
for col in numerical_cols:
    # split into categories
    category_dists = []
    for category in after_2010['SIG_SUB'].unique():
        dist = after_2010[col][after_2010['SIG_SUB'] == category]
        category_dists.append(dist)
    kw, p_value = kruskal(*category_dists)
    eta = pg.eta_squared(df, x='SIG_SUB', y=col)
    kruskal_results[col] = eta

AttributeError: module 'pingouin' has no attribute 'eta_squared'

In [91]:
kruskal_results

{'VIOLATOR_INSPECTION_DAY_CNT': 1.0748293711702565e-156,
 'VIOLATOR_VIOLATION_CNT': 8.044511197714421e-51,
 'YEAR_OCCUR': 0.0}

In [23]:
chi2_results

{'PRIMARY_OR_MILL': {'p-value': 1.1806487483045844e-137,
  'V': 0.02234434476076939,
  'stat': 630.5761873321704},
 'COAL_METAL_IND': {'p-value': 1.719884991446243e-109,
  'V': 0.01978149879948708,
  'stat': 494.2204445513551},
 'MINE_TYPE': {'p-value': 3.1298000262005726e-55,
  'V': 0.014097360203066883,
  'stat': 251.0024220031085},
 'VIOLATOR_TYPE_CD': {'p-value': 0.0,
  'V': 0.04731106779455914,
  'stat': 2827.013087581504}}

In [71]:
(x, y), (s, e, t) = violation_common.scale_and_encode(processed)

/home/zack/mambaforge/envs/general/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [117]:
after_2010.to_csv('after_2010.csv', index=False)

In [116]:
after_2010

,PRIMARY_OR_MILL,VIOLATOR_INSPECTION_DAY_CNT,COAL_METAL_IND,PROPOSED_PENALTY,VIOLATOR_VIOLATION_CNT,MINE_TYPE,SIG_SUB,VIOLATOR_TYPE_CD,YEAR_OCCUR
1000256,Not_Applicable,2847.0,C,100.0,410.0,Underground,N,Operator,2011.0
1000274,Not_Applicable,596.0,C,392.0,365.0,Underground,N,Operator,2011.0
1001616,Not_Applicable,22.0,C,873.0,0.0,Surface,Y,Operator,2011.0
1001707,Not_Applicable,11.0,C,100.0,0.0,Facility,N,Operator,2011.0
1001854,Primary,127.0,M,946.0,284.0,Facility,N,Operator,2011.0
...,...,...,...,...,...,...,...,...,...
2792267,Not_Applicable,552.0,C,155.0,179.0,Underground,N,Operator,2022.0
2792268,Not_Applicable,412.0,C,159.0,166.0,Underground,N,Operator,2021.0
2792269,Primary,170.0,M,5699.0,217.0,Surface,Y,Operator,2023.0
2792270,Primary,2.0,M,133.0,3.0,Surface,N,Operator,2022.0


In [118]:
pd.read_csv('after_2010.csv')

,PRIMARY_OR_MILL,VIOLATOR_INSPECTION_DAY_CNT,COAL_METAL_IND,PROPOSED_PENALTY,VIOLATOR_VIOLATION_CNT,MINE_TYPE,SIG_SUB,VIOLATOR_TYPE_CD,YEAR_OCCUR
0,Not_Applicable,2847.0,C,100.0,410.0,Underground,N,Operator,2011.0
1,Not_Applicable,596.0,C,392.0,365.0,Underground,N,Operator,2011.0
2,Not_Applicable,22.0,C,873.0,0.0,Surface,Y,Operator,2011.0
3,Not_Applicable,11.0,C,100.0,0.0,Facility,N,Operator,2011.0
4,Primary,127.0,M,946.0,284.0,Facility,N,Operator,2011.0
...,...,...,...,...,...,...,...,...,...
1262992,Not_Applicable,552.0,C,155.0,179.0,Underground,N,Operator,2022.0
1262993,Not_Applicable,412.0,C,159.0,166.0,Underground,N,Operator,2021.0
1262994,Primary,170.0,M,5699.0,217.0,Surface,Y,Operator,2023.0
1262995,Primary,2.0,M,133.0,3.0,Surface,N,Operator,2022.0
